In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
import psycopg2
from psycopg2 import extras  

In [ ]:
from faker import Faker
fake = Faker('en_IN')

In [ ]:
conn = psycopg2.connect(database="ecomm",
                        user='sushil', password='admin123', 
                        host='34.73.147.246', port='5432'
)

conn.autocommit = True
cursor = conn.cursor()

# Generate 5000 new Orders (incremental load)

In [ ]:
sql = '''select max(customerid) from customer_master;'''
cursor.execute(sql)
for i in cursor.fetchall():
    maxcustid=i
maxcustid = maxcustid[0]

In [ ]:
sql = '''select max(productid) from product_master;'''
cursor.execute(sql)
for i in cursor.fetchall():
    maxproductid=i
maxproductid = maxproductid[0]


In [ ]:
order_detail_fields = ['orderid','customerid','order_status_update_timestamp','order_status']

In [ ]:
orders_Received = pd.DataFrame(columns=order_detail_fields, index = range(20000,25000))
orders_InProgress = pd.DataFrame(columns=order_detail_fields, index = range(20000,25000))
orders_Delivered = pd.DataFrame(columns=order_detail_fields, index = range(20000,25000))

In [ ]:
for i in range(20000,25000):
    orders_Received['orderid'][i]=i
    orders_Received['customerid'][i]=fake.random_int(min=1, max=maxcustid)
    orders_Received['order_status_update_timestamp'][i]=fake.future_datetime('+30d')
    orders_Received['order_status']='Received'
    
for i in range(20000,25000):
    orders_InProgress['orderid'][i]=i
    orders_InProgress['customerid'][i]=orders_Received['customerid'][i]
    orders_InProgress['order_status_update_timestamp'][i]=orders_Received['order_status_update_timestamp'][i]+timedelta(seconds=fake.random_int(min=1, max=86400))
    orders_InProgress['order_status'][i]='InProgress'
    
for i in range(20000,25000):
    orders_Delivered['orderid'][i]=i
    orders_Delivered['customerid'][i]=orders_Received['customerid'][i]
    orders_Delivered['order_status_update_timestamp'][i]=orders_InProgress['order_status_update_timestamp'][i]+timedelta(seconds=fake.random_int(min=1, max=86400))
    orders_Delivered['order_status'][i]='Delivered'
    

In [ ]:
order_details_temp = orders_Received.append(orders_InProgress)
order_details = order_details_temp.append(orders_Delivered)
order_details.sort_values(by=['orderid','order_status_update_timestamp'])

In [ ]:
order_item_fields = ['orderid','productid','quantity']

In [ ]:
order_items = pd.DataFrame(columns=order_item_fields, index = range(1,5001))

In [ ]:
for i in range(1,5001):
    order_items['orderid'][i]=fake.random_element(order_details['orderid'])
    order_items['productid'][i]=fake.random_int(min=1, max=maxproductid)
    order_items['quantity'][i]=fake.random_int(min=1, max=20)

In [ ]:
order_items = order_items.drop_duplicates(subset=['orderid', 'productid'],keep='first')
order_items.sort_values(by=['orderid','productid'])

# Load incremental data into OLTP

In [ ]:
table = 'order_details'
df_columns = ",".join(order_detail_fields)
values = "VALUES({})".format(",".join(["%s" for _ in order_detail_fields])) 
insert_stmt = "INSERT INTO {} ({}) {}".format(table,df_columns,values)

psycopg2.extras.execute_batch(cursor, insert_stmt, order_details.values)

conn.commit()

In [ ]:
table = 'order_items'
df_columns = ",".join(order_item_fields)
values = "VALUES({})".format(",".join(["%s" for _ in order_item_fields])) 
insert_stmt = "INSERT INTO {} ({}) {}".format(table,df_columns,values)

psycopg2.extras.execute_batch(cursor, insert_stmt, order_items.values)

conn.commit()

# Validate

In [ ]:
sql1='''select count(1) from customer_master limit 50;'''

In [ ]:
sql2='''select count(1) from product_master  limit 50;'''

In [ ]:
sql3='''select count(1) from order_details  limit 50;'''

In [ ]:
sql4='''select count(1) from order_items limit 50;'''

In [ ]:
cursor.execute(sql3)
for i in cursor.fetchall():
    print(i)